# Unit Test Generator

The requirement: use a Frontier model to generate fast and repeatable unit tests for Python code.


In [ ]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-7-sonnet-20250219"

In [ ]:
system_message = "You are an assistant that implements unit testing for Python code. "
system_message += "Respond only with Python code; use comments sparingly and do not provide any explanation other than occasional comments. "

In [ ]:
def user_prompt_for(python_file):
    # Read the Python source file
    with open(python_file, "r", encoding="utf-8") as f:
        python_code = f.read()

    # Build the user prompt
    user_prompt = "Write unit tests for this Python code. "
    user_prompt += "Respond only with Python code; do not explain your work other than a few comments. "
    user_prompt += "The unit testsing is done in Jupyterlab, so you should use packages that play nicely with the Jupyter kernel."
    user_prompt += "Include the original Python code in your generated output so that I can run all in one fell swoop.\n\n"
    user_prompt += python_code
    
    return user_prompt

In [ ]:
def messages_for(python_file):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python_file)}
    ]

In [ ]:
def stream_gpt(python_file):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python_file), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [ ]:
def stream_claude(python_file):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python_file)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```python\n','').replace('```','')

In [ ]:
def unit_test(python_file, model):
    if model=="GPT":
        result = stream_gpt(python_file)
    elif model=="Claude":
        result = stream_claude(python_file)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("## Unit Test Generator\nUpload a Python file and get structured unit testing.")
    with gr.Row(): # Row 1
        orig_code = gr.File(label="Upload your Python file", file_types=[".py"])
        test_code = gr.Textbox(label="Unit test code:", lines=10)
    with gr.Row(): # Row 2
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row(): # Row 3
        generate = gr.Button("Generate unit test code")
    with gr.Row(): # Row 4
        unit_run = gr.Button("Run Python unit test")
    with gr.Row(): # Row 5
        test_out = gr.TextArea(label="Unit test result:", elem_classes=["test_code"])

    generate.click(unit_test, inputs=[orig_code, model], outputs=[test_code])
    unit_run.click(execute_python, inputs=[test_code], outputs=[test_out])

In [ ]:
ui.launch(inbrowser=True)